### Part 1

In [11]:
import json
import os, sys
from pprint import pprint
import numpy as np
import time

path = "data/"
dirs = os.listdir(path)

data = list([])
for file in dirs:
    temp = json.load(open(path+file))
    for i in temp:
        a = "body" in i.keys()
        b = "topics" in i.keys()
        if a and b:
            data.append(i)

cutoff = int(1.0/5 * len(data))            


In [17]:
word_key = list()
bag = list()

t1 = int(round(time.time() * 1000))

for d in data:
    row = [0 for i in range(len(word_key))]
    for w in d['body'].lower().split():
        if w not in word_key:
            word_key.append(w)
            row.append(0)
        i = word_key.index(w)
        row[i] += 1
    bag.append(row)

for d in bag:
    while len(d) < len(word_key):
        d.append(0)

t2 = int(round(time.time() * 1000))

print("Milliseconds used to create bag-of-words: ", t2-t1)

X_train = bag[0:cutoff]
X_test = bag[cutoff:-1]
y_train = [int('earn' in x["topics"]) for x in data[0:cutoff]]
y_test = [int('earn' in x["topics"]) for x in data[cutoff:-1]]

print(len(X_train[0]))

Milliseconds used to create bag-of-words:  563185
70793


In [13]:

from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification

def predictor(X_train, y_train, X_test, y_test):

    clf = RandomForestClassifier()
    clf.fit(X_train, y_train)
    RandomForestClassifier(n_estimators = 50)

    misclassification_count = sum([k*k for k in clf.predict(X_test)-y_test])
    misclassification_rate  = misclassification_count/len(y_test)

    return (1 - misclassification_rate)

In [14]:
%time print(predictor(X_train, y_train, X_test, y_test))

0.927237682207
CPU times: user 1min 9s, sys: 4.34 s, total: 1min 13s
Wall time: 1min 36s


### Part 2

In [5]:
import pymmh3

def hash_to_bucket(e, B):
    i = pymmh3.hash128(str(e))
    p = i / float(2**128)
    for j in range(0, B):
        if j/float(B) <= p and (j+1)/float(B) > p:
            return j+1
    return B


In [15]:

t1 = int(round(time.time() * 1000))

y_train = [int('earn' in x["topics"]) for x in data[0:cutoff]]
y_test = [int('earn' in x["topics"]) for x in data[cutoff:-1]]

M = [[hash_to_bucket(w, 1000) for w in a['body'].lower().split()] for a in data]
X = [[0 for i in range(1000)] for j in range(len(data))]
for i in range(len(data)):
    for j in M[i]:
        X[i][j-1] += 1

t2 = int(round(time.time() * 1000))

print("Milliseconds used to create hash table: ", t2-t1)



Milliseconds used to create hash table:  576163


In [16]:

X_train =  X[0:cutoff]
X_test = X[cutoff:-1]

print(predictor(X_train, y_train, X_test, y_test))


0.920130104807
